In [1]:
import pandas as pd
import duckdb
from datasetsforecast.m5 import M5

# Get all returned values from M5.load()
walmart_orders_df, walmart_products_df, store_reference_df = M5.load(directory='data')


print(f"Number of unique IDs in train_df: {walmart_orders_df['unique_id'].nunique()}")

walmart_orders_df.info()
print(walmart_orders_df.head())
print(walmart_orders_df.describe())

walmart_products_df.info()
print(walmart_products_df.head())
print(walmart_products_df.describe())

store_reference_df.info()
print(store_reference_df.head())
print(store_reference_df.describe())

#foods_ids = walmart_orders_df[walmart_orders_df['unique_id'].str.startswith('FOODS', na=False)]

# Get the count of FOODS ids
#print(f"Number of unique IDs starting with 'FOODS': {foods_ids['unique_id'].nunique()}")

# Show the first few examples
#print("\nSample FOODS IDs:")
#print(foods_ids['unique_id'].unique()[:10])

# Optional: Get a distribution of these IDs
#print("\nDistribution of FOODS IDs by category (if applicable):")
#if 'category' in foods_ids.columns:
#    print(foods_ids.groupby('category').size())





Number of unique IDs in train_df: 30490
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47649940 entries, 0 to 47649939
Data columns (total 3 columns):
 #   Column     Dtype         
---  ------     -----         
 0   unique_id  category      
 1   ds         datetime64[ns]
 2   y          float32       
dtypes: category(1), datetime64[ns](1), float32(1)
memory usage: 636.4 MB
          unique_id         ds    y
0  FOODS_1_001_CA_1 2011-01-29  3.0
1  FOODS_1_001_CA_1 2011-01-30  0.0
2  FOODS_1_001_CA_1 2011-01-31  0.0
3  FOODS_1_001_CA_1 2011-02-01  1.0
4  FOODS_1_001_CA_1 2011-02-02  4.0
                                  ds             y
count                       47649940  4.764994e+07
mean   2014-02-16 06:00:56.859065856  1.430355e+00
min              2011-01-29 00:00:00  0.000000e+00
25%              2012-12-14 00:00:00  0.000000e+00
50%              2014-04-11 00:00:00  0.000000e+00
75%              2015-05-25 00:00:00  1.000000e+00
max              2016-06-19 00:00:00  7.6300

In [21]:
# Calculate average daily sales for each product
# First, group by unique_id and calculate the mean of sales
avg_sales_per_product = walmart_orders_df.groupby('unique_id', observed=False)['y'].mean()

# Find products with average daily sales around 1000 (within a range)
# You can adjust the threshold based on how close to 1000 you want
threshold = 100  # This means looking for products with avg sales between 900-1100
target_products = avg_sales_per_product[(avg_sales_per_product > 1000 - threshold) &
                                        (avg_sales_per_product < 1000 + threshold)]


# Sort them by how close they are to 1000
target_products = target_products.sort_values(key=lambda x: abs(x - 1000))

threshold = 100  # This means looking for products with avg sales between 900-1100
target_products = avg_sales_per_product[(avg_sales_per_product > 1000 - threshold) &
                                        (avg_sales_per_product < 1000 + threshold)]


# Display the results
print(f"Found {len(target_products)} products with average daily sales around 1000 (±{threshold})")
print("\nTop 10 products closest to 1000 average daily sales:")
for product_id, avg_sales in target_products.head(10).items():
    print(f"Product ID: {product_id}, Average Daily Sales: {avg_sales:.2f}")

# If you specifically want FOODS products around 1000 sales
foods_avg_sales = avg_sales_per_product[avg_sales_per_product.index.str.startswith('FOODS')]
closest_idx_position = (foods_avg_sales - 1000).abs().argsort().iloc[0]

# Use .iloc to access by position
closest_food_product = foods_avg_sales.iloc[closest_idx_position]
closest_food_id = foods_avg_sales.index[closest_idx_position]

print(f"\nClosest FOODS product to 1000 avg sales: {closest_food_id} with {closest_food_product:.2f} avg sales")



Found 0 products with average daily sales around 1000 (±100)

Top 10 products closest to 1000 average daily sales:

Closest FOODS product to 1000 avg sales: FOODS_3_090_CA_3 with 130.68 avg sales
